In [ ]:
# import tensorflow module. Check API version.
import tensorflow as tf
import numpy as np

print (tf.__version__)

# required for TF to run within docker using GPU (ignore otherwise)
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [ ]:
# grab the CIFAR-10 dataset (may take time the first time)
print("[INFO] downloading CIFAR-10...")
((trainData, trainLabels), (testData, testLabels)) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# parameters for CIFAR-10 data set
num_classes = 10
image_width = 32
image_height = 32
image_channels = 3 # CIFAR data is RGB color
# define human readable class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# shape the input data using "channels last" ordering
# num_samples x rows x columns x depth
trainData = trainData.reshape(
        (trainData.shape[0], image_height, image_width, image_channels))
testData = testData.reshape(
        (testData.shape[0], image_height, image_width, image_channels))

In [ ]:
# convert to floating point and scale data to the range of [0.0, 1.0]
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

In [ ]:
# resize the data to 96X96 for use in inceptionV3 network
trainData = tf.image.resize(trainData, [96, 96], 'bicubic')
testData = tf.image.resize(testData, [96, 96], 'bicubic')

In [ ]:
# display data dimentions
print ("trainData:", trainData.shape)
print ("trainLabels:", trainLabels.shape)
print ("testData:", testData.shape)
print ("testLabels:", testLabels.shape)

In [ ]:
# parameters for training data set
num_classes = 10
image_width = 96
image_height = 96
image_channels = 3

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (96, 96, 3),
                                include_top = False,
                                weights = None)

pre_trained_weights_file = '../../data/deep_learning/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model.load_weights(pre_trained_weights_file)

pre_trained_model.trainable = False

last_output = pre_trained_model.get_layer('mixed7').output

base_model = tf.keras.Model(pre_trained_model.input, last_output)

base_model.summary()

In [ ]:
# initialize the model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=num_classes, activation=tf.nn.softmax)
])

# compile the model
print("[INFO] compiling model...")
model.compile(optimizer=tf.keras.optimizers.Adam(),  # Adam optimizer
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])

# print model summary
model.summary()

In [ ]:
# define callback function for training termination criteria
#accuracy_cutoff = 0.99
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if(logs.get('val_accuracy') > 0.90):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

# initialize training config
batch_size = 16
epochs = 100

# run training
print("[INFO] training...")
history = model.fit(x=trainData, y=trainLabels, batch_size=batch_size,
                    validation_data=(testData, testLabels), epochs=epochs, verbose=1, callbacks=[myCallback()])